# Feature extraction from preprocessed data
Relevant features are extracted directly from the relations data

In [1]:
import numpy as np
import pandas as pd
import csv
from matplotlib import pyplot as plt
import time
import sys
import os
import subprocess
import datetime
import math

def restart_line():
    sys.stdout.write('\r')
    sys.stdout.flush()

filenames =['relations_extended.csv']

path = "/Users/jonasmuller/Local_Folder/Project NTDS/"
file = path + filenames[0]

In [2]:
chunksize = 1000000
i = 0
carry = pd.DataFrame()
df_buffer = pd.DataFrame()
features = pd.DataFrame()
rel_dist = pd.DataFrame(np.zeros(7),list(range(1,8)),columns=["count"])
t1 = time.time()
for chunk in pd.read_csv(file, chunksize=chunksize, sep = "\t", names=["day", "time_ms", "src", "dst","relation"]):
    
    # slice chunk at specific src
    chunk = chunk.append(carry)
    
    # remove last source in chunk
    last_src = np.unique(chunk["src"].values)[-1]
    carry = chunk.loc[chunk["src"] == last_src]
    
    chunk = chunk.loc[chunk["src"] != last_src]
    sources_in_chunk = np.unique(chunk["src"].values)
    
    # count number of out degrees
    degree_tot = chunk.groupby('src').dst.count()
    degree_out = chunk.loc[chunk["relation"] < 10].groupby('src').dst.count()
    degree_in = chunk.loc[chunk["relation"] > 10].groupby('src').dst.count()
    
    # count number of unique neighbors
    unique_neighbors = chunk.groupby('src').dst.nunique()
    
    # bidirectional relations
    # intersection between in and out relation nodes
    
    # weights acording to relation scarcity (then deg in deg out) / number of strong connections per node
    
    # relation distribution count
    values, counts = np.unique(chunk.loc[chunk["relation"] < 10].relation.values, return_counts=True)
    rel_dist = rel_dist.add(pd.Series(counts, index=values),axis='index')
    
    # append degrees
    df_buffer['deg_tot'] = pd.Series(degree_tot)
    df_buffer['deg_out'] = pd.Series(degree_out)
    df_buffer['deg_in'] = pd.Series(degree_in)
    df_buffer['uni_neigh'] = pd.Series(unique_neighbors)
    
    features = features.append(df_buffer)
    
    # break for debugging
    i = i+1
    if i < 1:
        break
        
    restart_line()
    sys.stdout.write(f'Chunk Nr: {i}; feature shhape: {features.shape}')
    sys.stdout.flush()

# print relevant key data
m = math.floor((time.time()-t1)/60)
s = (time.time()-t1)-m*60
print(f'\nExecution time {m} min {s:.2f} sec')
print(f'number of cycles {i}')

# statistics
total_relations = np.sum(rel_dist.values)
rel_dist = rel_dist.div(total_relations)
with open("Output.txt", "w") as text_file:
    print(f"Nr of total relations: {total_relations}\n", file=text_file)
    print(rel_dist, file=text_file)

Chunk Nr: 1717; feature shhape: (5399965, 4)
Execution time 23 min 16.31 sec
number of cycles 1717


In [ ]:
# visualize
ax = plt.figure(1, figsize=(15, 3))
plt.title("DegreeOut histogram")
plt.hist(dout.values, bins=200);
plt.ylabel('Number of Nodes')
plt.xlabel('Degree Out')

In [3]:
# save features
exportName = filenames[0][:-4] + "_features.csv"
print(exportName)
features.to_csv(exportName, sep = '\t')

relations_extended_features.csv
